## 환경설정

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# 한글 폰트 설정 (Windows)
rcParams['font.family'] = 'Malgun Gothic'  # Windows 기본 한글 폰트
rcParams['axes.unicode_minus'] = False     # 마이너스 깨짐 방지

from __future__ import annotations

#라이브러리
from typing import Dict, Optional, Sequence
import numpy as np
import pandas as pd

## 추천시스템

In [ ]:
"""
천안 복합작물 추천시스템 (단일 + 페어/타깃 기반)
----------------------------------------------

요구사항 반영 버전 (top_n 지원 강화):
- **단일 작물 추천**: 가격 PKL **미사용**. `df_features`만 사용. → **top_n 파라미터 추가**로 상위 N개만 반환 가능.
- **복합(페어) 추천**: 가격 PKL은 **오직** 두 작물 **가격시계열 합의 CV**(`price_cv_sum`) 계산에만 사용. (제공 PKL은 **단일 파일(5년치)**) → **top_n**이 정렬 후 정확히 적용되도록 수정.
- **price_rank 사용**
  - 단일: `annual_pred_rev`를 랭크화한 **`price_rank`를 점수에 직접 사용**(작을수록 좋음 → 역방향 적용)
  - 페어: `annual_pred_rev_sum`을 랭크화한 **`price_rank_pair`를 점수에 직접 사용**(작을수록 좋음 → 역방향 적용)
- **가중치**
  - 단일: 5개 지표 × 0.2 (고정)
  - 페어: 5개 지표 × 가중치 사용자 설정(디폴트 0.2씩)
- **정규화 규칙**
  - 가격 관련은 랭킹(`price_rank`, `price_rank_pair`), 그 외는 min-max
  - **작을수록 좋은 컬럼**: 단일(`price_cv`, `price_rank`, `pest_count`), 페어(`price_cv_sum`, `price_rank_pair`, `pest_count_sum`)
    → 점수 계산 시 **역방향(1-x)** 처리

스키마 고정(df_features)
------------------------
['itemcode', 'crop_name', 'annual_pred_rev', 'price_cv', 'env_sensitivity', 'suitable_days', 'pest_count']

"""


def _to_float_price(s: pd.Series) -> pd.Series:
    """가격 시리즈를 float로 변환
    - 정수/실수형이면 그대로 float 캐스팅
    - 문자열인 경우 콤마/공백 제거 후 float 변환
    - 변환 불가한 값은 NaN 처리
    """
    if s.dtype.kind in {"i", "u", "f"}:
        return s.astype(float)
    return (
        s.astype(str)
         .str.replace(",", "", regex=False)
         .str.replace(" ", "", regex=False)
         .replace({"": np.nan})
         .astype(float)
    )


def _minmax(x: pd.Series) -> pd.Series:
    """min-max 정규화 (0~1). 상수열인 경우 0으로 채움"""
    x = pd.to_numeric(x, errors="coerce")
    xmin, xmax = np.nanmin(x), np.nanmax(x)
    if not np.isfinite(xmin) or not np.isfinite(xmax) or xmax == xmin:
        return pd.Series(np.zeros(len(x)), index=x.index, dtype=float)
    return (x - xmin) / (xmax - xmin)


def _rank01(x: pd.Series, ascending: bool = True) -> pd.Series:
    """랭크를 0~1로 변환
    - ascending=True  : 값이 **작을수록** 좋은 랭크(0에 가까움)
    - ascending=False : 값이 **클수록** 좋은 랭크(0에 가까움)  ← 매출 랭크 계산에 사용
    - 동순위는 평균 순위 사용, 유효값 1개 이하면 0으로 처리
    """
    ranks = x.rank(method="average", na_option="keep", ascending=ascending)
    n = max(1, x.notna().sum() - 1)
    return (ranks - 1) / n


def _invert01(x01: pd.Series) -> pd.Series:
    """0~1 값에 대해 역방향(작을수록 좋음)으로 변환: 1 - x"""
    return 1.0 - x01


def _cv(series: pd.Series) -> float:
    """시리즈의 변동계수(CV = 표준편차/평균) 계산. 평균 0 또는 NaN이면 NaN 반환"""
    mu = series.mean()
    sd = series.std()
    if not np.isfinite(mu) or mu == 0:
        return float("nan")
    return float(sd / mu)


# =====================================
# 가격 데이터 로딩/리샘플 (단일 PKL 입력) — 페어 추천에서만 사용
# =====================================

def load_price_pkl(pkl_path: str) -> pd.DataFrame:
    """**단일** PKL 파일(예: 5년치)을 로드하여 표준 스키마로 반환
    - 필요한 컬럼: date, itemcode, kindcode, price
    - 출력 컬럼: ['date','itemcode','kindcode','price']
    - (주의) 단일 추천은 PKL을 사용하지 않음. 페어 추천의 합시계열 CV 계산에만 사용.
    """
    df = pd.read_pickle(pkl_path)
    cols = {c.lower(): c for c in df.columns}
    date_col = cols.get("date", "date")
    item_col = cols.get("itemcode", "itemcode")
    kind_col = cols.get("kindcode", "kindcode")
    price_col = cols.get("price", "price")
    out = pd.DataFrame({
        "date": pd.to_datetime(df[date_col], errors="coerce"),
        "itemcode": df[item_col].astype(str),
        "kindcode": df[kind_col].astype(str),
        "price": _to_float_price(df[price_col])
    })
    out = out.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
    return out[["date","itemcode","kindcode","price"]]


def resample_prices(df_prices: pd.DataFrame, key_col: str = "itemcode", freq: str = "M", how: str = "mean") -> pd.DataFrame:
    """가격 시계열을 주기별로 리샘플하여 피벗(열=작물코드) 형태로 반환
    - 페어 추천의 가격 합시계열(CV) 계산을 위해 사용
    - 결측은 ffill/bfill 후 컬럼 평균으로 보강, 최종적으로 남는 NaN은 0 처리
    """
    df = df_prices[["date", key_col, "price"]].dropna()
    wide = df.groupby([pd.Grouper(key="date", freq=freq), key_col])["price"].agg(how).unstack(key_col)
    wide = wide.sort_index()
    wide = wide.ffill().bfill()
    col_means = wide.mean(axis=0, skipna=True)
    wide = wide.fillna(col_means)
    wide = wide.fillna(0.0)
    return wide


# =====================
# 단일 작물 추천 (고정, PKL 미사용)
# =====================

def single_crop_ranking(
    df_features: pd.DataFrame,
    use_price_rank: bool = True,  # 호환성 파라미터(항상 price_rank 사용)
    top_n: Optional[int] = None,  # 추가: 상위 N개만 반환
) -> pd.DataFrame:
    """단일 작물 추천 점수 계산 (가중치 고정: 0.2×5)
    - 사용 지표(고정 5개): **price_rank(annual_pred_rev 랭크)**, price_cv, env_sensitivity, suitable_days, pest_count
    - 정규화 규칙:
      * price_rank → 이미 0~1(작을수록 좋음)이므로 **역방향 적용** 후 사용
      * price_cv → min-max 후 역방향(작을수록 좋음)
      * env_sensitivity → min-max (클수록 좋음)
      * suitable_days → min-max (클수록 좋음)
      * pest_count → min-max 후 역방향(작을수록 좋음)
    - (중요) 가격 PKL은 사용하지 않음
    - (신규) top_n 지정 시 상위 N개만 반환
    """
    required = ["itemcode","crop_name","annual_pred_rev","price_cv","env_sensitivity","suitable_days","pest_count"]
    missing = [c for c in required if c not in df_features.columns]
    if missing:
        raise ValueError(f"df_features에 필요한 컬럼이 없습니다: {missing}")

    df = df_features.copy()
    df["itemcode"] = df["itemcode"].astype(str)

    # price_rank 생성: annual_pred_rev를 기준으로 높은 매출일수록 0에 가깝게(작을수록 좋음)
    price_rank = _rank01(df["annual_pred_rev"], ascending=False)

    # 정규화 값 계산 (0~1, 모두 **클수록 좋음**으로 맞추기 위해 작은 값 선호 항목은 역방향 적용)
    price_rank_n = _invert01(price_rank)                     # 작은 값 선호 → 역방향
    cv_n         = _invert01(_minmax(df["price_cv"]))       # 작은 값 선호 → 역방향
    env_n        = _minmax(df["env_sensitivity"])           # 큰 값 선호
    days_n       = _minmax(df["suitable_days"])             # 큰 값 선호
    pest_n       = _invert01(_minmax(df["pest_count"]))     # 작은 값 선호 → 역방향

    # 고정 가중치 0.2씩 (총 1.0)
    w = 0.2
    score = w*price_rank_n + w*cv_n + w*env_n + w*days_n + w*pest_n

    out = pd.DataFrame({
        "itemcode": df["itemcode"],
        "crop_name": df["crop_name"],
        "score_single": score,
        # 리포팅용 세부 지표
        "price_rank": price_rank,
        "price_rank_n": price_rank_n,
        "price_cv_n": cv_n,
        "env_sensitivity_n": env_n,
        "suitable_days_n": days_n,
        "pest_count_n": pest_n,
    })

    out = out.sort_values("score_single", ascending=False).reset_index(drop=True)
    if top_n is not None:
        out = out.head(int(top_n))
    return out


# =====================
# 페어 추천 (타깃 기반, PKL 사용)
# =====================

def _resolve_target_code(df_features: pd.DataFrame, target: str, key_col: str = "itemcode", name_col: str = "crop_name") -> str:
    """사용자가 입력한 타깃(코드 또는 이름)을 itemcode 문자열로 해석
    - 우선 itemcode 완전일치 탐색 → 없으면 crop_name 완전일치(대소문자 무시)
    - crop_name 중복 시 에러 발생
    """
    target = str(target).strip()
    if target in df_features[key_col].astype(str).tolist():
        return target
    hits = df_features[df_features[name_col].astype(str).str.strip().str.lower() == target.lower()]
    if len(hits) == 1:
        return str(hits.iloc[0][key_col])
    elif len(hits) > 1:
        dup = ", ".join(hits[key_col].astype(str).tolist())
        raise ValueError(f"동일 crop_name이 여러 개입니다. itemcode로 지정하세요. 후보: {dup}")
    else:
        raise ValueError("타깃을 찾을 수 없습니다. itemcode 또는 crop_name을 확인하세요.")


def pair_recommendation_for_target(
    df_prices: pd.DataFrame,
    df_features: pd.DataFrame,
    target: str,  # itemcode 또는 crop_name
    key_col: str = "itemcode",
    name_col: str = "crop_name",
    freq: str = "M",
    weights: Optional[Dict[str, float]] = None,
    top_n: Optional[int] = 10,
) -> pd.DataFrame:
    """타깃 작물을 넣으면 해당 타깃과의 조합이 좋은 파트너들을 순위로 반환
    - 사용 지표(5개):
      1) **price_rank_pair** = annual_pred_rev_sum의 랭크(0~1, 작을수록 좋음)  ← 점수에 사용
      2) price_cv_sum        = 두 작물 **가격시계열 합**의 CV (작을수록 좋음)
      3) env_sensitivity_sum = 두 작물 env_sensitivity 합 (클수록 좋음)
      4) suitable_days_sum   = 두 작물 suitable_days 합 (클수록 좋음)
      5) pest_count_sum      = 두 작물 pest_count 합 (작을수록 좋음)
    - 정규화: 작은 값 선호 항목은 역방향 처리 후 가중합
    - 가중치: 미지정 시 각 0.2 (키는 아래 *_n 열 기준)
    - (중요) 가격 PKL은 price_cv_sum 계산에만 사용
    - (신규) top_n 지정 시 상위 N개만 반환 (정렬 후 head 적용)
    """
    if weights is None:
        weights = {
            "price_rank_pair_n": 0.2,
            "price_cv_sum_n": 0.2,
            "env_sensitivity_sum_n": 0.2,
            "suitable_days_sum_n": 0.2,
            "pest_count_sum_n": 0.2,
        }

    # 타깃 해석
    target_code = _resolve_target_code(df_features, target, key_col=key_col, name_col=name_col)

    # 가격 wide 피벗 (CV 합계 계산 목적)
    wide = resample_prices(df_prices, key_col=key_col, freq=freq, how="mean")
    if target_code not in wide.columns.astype(str).tolist():
        raise ValueError("가격 테이블에 타깃 코드가 없습니다.")

    # 특성 인덱스 준비
    req = ["itemcode","crop_name","annual_pred_rev","price_cv","env_sensitivity","suitable_days","pest_count"]
    missing = [c for c in req if c not in df_features.columns]
    if missing:
        raise ValueError(f"df_features에 필요한 컬럼이 없습니다: {missing}")
    feat = df_features.copy()
    feat[key_col] = feat[key_col].astype(str)
    feat = feat.set_index(key_col)

    sT = wide[str(target_code)].dropna()

    rows = []
    codes = [str(c) for c in wide.columns if str(c) in feat.index and str(c) != str(target_code)]
    for b in codes:
        sB = wide[b].dropna()
        common = sT.index.intersection(sB.index)
        if len(common) < 3:
            continue
        T = sT.loc[common]
        B = sB.loc[common]

        # 가격 합시계열 CV
        price_cv_sum = _cv(T + B)

        # 매출 합
        annual_pred_rev_sum = float(feat.loc[target_code, "annual_pred_rev"]) + float(feat.loc[b, "annual_pred_rev"])

        rows.append({
            "partner_code": b,
            "partner_name": str(feat.loc[b, name_col]) if name_col in feat.columns else None,
            "price_cv_sum": price_cv_sum,
            "env_sensitivity_sum": float(feat.loc[target_code, "env_sensitivity"]) + float(feat.loc[b, "env_sensitivity"]),
            "suitable_days_sum":   float(feat.loc[target_code, "suitable_days"])   + float(feat.loc[b, "suitable_days"]),
            "pest_count_sum":      float(feat.loc[target_code, "pest_count"])      + float(feat.loc[b, "pest_count"]),
            "annual_pred_rev_sum": annual_pred_rev_sum,
        })

    pairs = pd.DataFrame(rows)
    if pairs.empty:
        return pairs

    # price_rank_pair 계산: annual_pred_rev_sum 기준, 큰 값이 좋은 것이므로 ascending=False로 0에 가깝게
    pairs["price_rank_pair"] = _rank01(pairs["annual_pred_rev_sum"], ascending=False)

    # 정규화/역방향 적용 (모두 최종적으로 **클수록 좋음** 스케일로 정렬)
    pairs["price_rank_pair_n"]    = _invert01(pairs["price_rank_pair"])             # 작은 값 선호 → 역방향
    pairs["price_cv_sum_n"]       = _invert01(_minmax(pairs["price_cv_sum"]))       # 작은 값 선호 → 역방향
    pairs["env_sensitivity_sum_n"] = _minmax(pairs["env_sensitivity_sum"])          # 큰 값 선호
    pairs["suitable_days_sum_n"]   = _minmax(pairs["suitable_days_sum"])            # 큰 값 선호
    pairs["pest_count_sum_n"]      = _invert01(_minmax(pairs["pest_count_sum"]))    # 작은 값 선호 → 역방향

    # 가중합 점수
    def _lin_score(r: pd.Series) -> float:
        total = 0.0
        for k, w in weights.items():
            v = r.get(k, np.nan)
            if pd.isna(v):
                continue
            total += float(w) * float(v)
        return total

    pairs["pair_score"] = pairs.apply(_lin_score, axis=1)

    # 타깃 메타 병합 후 정렬/출력
    target_name = str(feat.loc[target_code, name_col]) if name_col in feat.columns else None
    pairs.insert(0, "target_code", target_code)
    pairs.insert(1, "target_name", target_name)

    cols = [
        "target_code","target_name","partner_code","partner_name","pair_score",
        "price_rank_pair","price_rank_pair_n",
        "annual_pred_rev_sum",
        "price_cv_sum","price_cv_sum_n",
        "env_sensitivity_sum","env_sensitivity_sum_n",
        "suitable_days_sum","suitable_days_sum_n",
        "pest_count_sum","pest_count_sum_n",
    ]

    pairs = pairs.sort_values("pair_score", ascending=False).reset_index(drop=True)
    if top_n is not None:
        pairs = pairs.head(int(top_n))
    return pairs[cols]

In [ ]:
#단일 추천 실행
df_features = pd.read_csv('chart_for_system.csv')
single_rank = single_crop_ranking(df_features, use_price_rank=True, top_n=5)
print("[단일 작물 추천 상위]")
print(single_rank)
single_rank

FileNotFoundError: [Errno 2] No such file or directory: 'chart_for_system.csv'

In [ ]:
# 페어 추천
df_features = pd.read_csv('chart_for_system.csv')
price_df = load_price_pkl("df_total_for_system.pkl")

pair_rank_tgt = pair_recommendation_for_target(
    price_df,
    df_features,
    target="토마토",             # target="414" or "포도"
    key_col="itemcode",
    name_col="crop_name",
    freq="M",
    weights = {
    "price_rank_pair_n": 0.5,
    "price_cv_sum_n": 0.1,
    "env_sensitivity_sum_n": 0.5,
    "suitable_days_sum_n": 0.1,
    "pest_count_sum_n": 0.1},
    top_n=5)
print("[페어 추천: 타깃=포도(코드 '414')]")
print(pair_rank_tgt[['target_code', "target_name", "partner_code",'partner_name', "pair_score"]])

## 시각화

In [ ]:
# =====================================
# 레이더 차트 유틸 (matplotlib 기반)
# =====================================

# (참고) 레이더 차트 축 이름: 단일과 페어 모두 동일한 5축을 사용
_SINGLE_AXES = [
    "price_rank_n",           # annual_pred_rev 랭크(역방향)
    "price_cv_n",             # price_cv 역방향
    "env_sensitivity_n",      # env_sensitivity 정규화
    "suitable_days_n",        # suitable_days 정규화
    "pest_count_n",           # pest_count 역방향
]
_SINGLE_AXES_LABEL = [
    "가격랭크(역)", "CV(역)", "환경민감", "적합일수", "병해충(역)"
]

_PAIR_AXES = [
    "price_rank_pair_n",      # annual_pred_rev_sum 랭크(역방향)
    "price_cv_sum_n",         # 합시계열 CV 역방향
    "env_sensitivity_sum_n",  # 합 환경민감
    "suitable_days_sum_n",    # 합 적합일수
    "pest_count_sum_n",       # 합 병해충(역)
]
_PAIR_AXES_LABEL = [
    "합-가격랭크(역)", "합-CV(역)", "합-환경민감", "합-적합일수", "합-병해충(역)"
]

def _make_polar_axes(n_vars: int):
    """레이더 차트용 극좌표 축과 각도 배열 생성
    - 시작 각도를 위쪽(π/2)으로 두고 시계방향으로 진행
    """
    angles = np.linspace(0, 2*np.pi, n_vars, endpoint=False).tolist()
    angles += angles[:1]  # 폴리곤 닫기
    ax = plt.subplot(111, polar=True)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.degrees(angles[:-1]))
    ax.set_rlabel_position(0)
    ax.set_ylim(0, 1)
    return ax, angles

# -------------------------------------------------
# (신규) 단일: 결과 표를 받아 **작물별로 하나씩** 레이더 저장/표시
# -------------------------------------------------

def plot_single_radars_from_result(
    single_df: pd.DataFrame,
    key_col: str = "itemcode",
    name_col: str = "crop_name",
    title_fmt: str = "단일 레이더 - {name}({code}) \n score={score:.3f}",
    out_dir: str = None,
    show: bool = True,
    per_row: int = 5,                # <- 추가: 한 행에 몇 개
    out_name: str = "single_grid.png"
):
    n = len(single_df)
    if n == 0:
        raise ValueError("입력된 single_df가 비었습니다.")

    cols = max(1, int(per_row))
    rows = int(np.ceil(n / cols))

    fig, axes = plt.subplots(rows, cols, subplot_kw={'projection': 'polar'},
                             figsize=(cols*3, rows*3))
    axes = np.atleast_1d(axes).ravel()

    # 각도/라벨 셋업(공통)
    angles = np.linspace(0, 2*np.pi, len(_SINGLE_AXES), endpoint=False).tolist()
    angles += angles[:1]

    for i, (_, row) in enumerate(single_df.iterrows()):
        ax = axes[i]
        # 극좌표 기본 설정
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_thetagrids(np.degrees(angles[:-1]), labels=_SINGLE_AXES_LABEL)
        ax.set_rlabel_position(0)
        ax.set_ylim(0, 1)

        code = str(row.get(key_col, ""))
        name = str(row.get(name_col, ""))
        score = float(row.get("score_single", np.nan))
        values = [row[a] for a in _SINGLE_AXES] + [row[_SINGLE_AXES[0]]]

        ax.set_title(title_fmt.format(name=name, code=code, score=score))
        ax.plot(angles, values)
        ax.fill(angles, values, alpha=0.1)

    # 남는 칸 비활성화
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    if out_dir:
        plt.savefig(f"{out_dir.rstrip('/')}/{out_name}", bbox_inches="tight")
    if show:
        plt.show()
    plt.close()


# -------------------------------------------------
# (신규) 페어: **기존에 계산된 pair_rank_tgt** 표를 받아
#              파트너별로 하나씩 레이더 저장/표시
# -------------------------------------------------

def plot_pair_radars_from_result(
    pair_df: pd.DataFrame,
    title_fmt: str = "페어 레이더 - {tname}({tcode}) × {pname}({pcode}) \n score={score:.3f}",
    out_dir: str = None,
    show: bool = True,
    per_row: int = 5,                # <- 추가: 한 행에 몇 개
    out_name: str = None             # 기본 파일명 자동 생성
):
    n = len(pair_df)
    if n == 0:
        raise ValueError("입력된 pair_df가 비었습니다.")

    cols = max(1, int(per_row))
    rows = int(np.ceil(n / cols))

    fig, axes = plt.subplots(rows, cols, subplot_kw={'projection': 'polar'},
                             figsize=(cols*3, rows*3))
    axes = np.atleast_1d(axes).ravel()

    # 각도/라벨 셋업(공통)
    angles = np.linspace(0, 2*np.pi, len(_PAIR_AXES), endpoint=False).tolist()
    angles += angles[:1]

    # 타깃 메타(파일명용)
    tcode = str(pair_df.iloc[0].get("target_code", ""))
    tname = str(pair_df.iloc[0].get("target_name", ""))

    for i, (_, row) in enumerate(pair_df.iterrows()):
        ax = axes[i]
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_thetagrids(np.degrees(angles[:-1]), labels=_PAIR_AXES_LABEL)
        ax.set_rlabel_position(0)
        ax.set_ylim(0, 1)

        pcode = str(row.get("partner_code", ""))
        pname = str(row.get("partner_name", ""))
        score = float(row.get("pair_score", np.nan))
        values = [row[a] for a in _PAIR_AXES] + [row[_PAIR_AXES[0]]]

        ax.set_title(title_fmt.format(tname=tname, tcode=tcode, pname=pname, pcode=pcode, score=score))
        ax.plot(angles, values)
        ax.fill(angles, values, alpha=0.1)

    # 남는 칸 비활성화
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    if out_dir:
        if out_name is None:
            out_name = f"pair_grid_{tcode}.png"
        plt.savefig(f"{out_dir.rstrip('/')}/{out_name}", bbox_inches="tight")
    if show:
        plt.show()
    plt.close()


In [ ]:
# 단일 레이더차트
plot_single_radars_from_result(single_rank, out_dir=None, show=True)

In [ ]:
# 페어 레이더차트
plot_pair_radars_from_result(pair_rank_tgt, out_dir=None, show=True)

## 대표 시나리오

In [ ]:
# 1) “가격 변동이 큰 딸기 운영자” -> 안정성 최우선
weights = {
  "price_rank_pair_n": 0.05,
  "price_cv_sum_n":    0.80,
  "env_sensitivity_sum_n": 0.05,
  "suitable_days_sum_n":   0.05,
  "pest_count_sum_n":      0.05}

pair_rank_tgt = pair_recommendation_for_target(
    price_df, df_features,
    target="딸기",
    key_col="itemcode", name_col="crop_name",
    freq="M",
    weights=weights,
    top_n=3
)
print(pair_rank_tgt)
plot_pair_radars_from_result(pair_rank_tgt, per_row=5, out_dir=None, show=True)

In [ ]:
# 2) “이미 매우 안정적인 작물 (들깨) ” -> 매출 업
weights = {
  "price_rank_pair_n": 0.70,
  "price_cv_sum_n":    0.15,
  "env_sensitivity_sum_n": 0.05,
  "suitable_days_sum_n":   0.05,
  "pest_count_sum_n":      0.05,
}


pair_rank_tgt = pair_recommendation_for_target(
    price_df, df_features,
    target="들깨",
    key_col="itemcode", name_col="crop_name",
    freq="M",
    weights=weights,
    top_n=3
)
print(pair_rank_tgt)
plot_pair_radars_from_result(pair_rank_tgt, per_row=3, out_dir=None, show=True)

In [ ]:
# 3) “ 스마트팜 운영자의 노지 구매 ” -> 적합일수 중요
weights = {
  "price_rank_pair_n": 0.15,
  "price_cv_sum_n":    0.05,
  "env_sensitivity_sum_n": 0.15,
  "suitable_days_sum_n":   0.50,
  "pest_count_sum_n":      0.15,
}


pair_rank_tgt = pair_recommendation_for_target(
    price_df, df_features,
    target="피마늘",
    key_col="itemcode", name_col="crop_name",
    freq="M",
    weights=weights,
    top_n=3
)
print(pair_rank_tgt)
plot_pair_radars_from_result(pair_rank_tgt, per_row=5, out_dir=None, show=True)